In [ ]:
import numpy as np
import pandas as pd
import scipy.ndimage as ndi
import matplotlib.pyplot as plt

import os
import time
import matplotlib

# import skimage
from skimage import io, filters, exposure, measure, transform
from scipy.signal import find_peaks, savgol_filter

pd.set_option('mode.chained_assignment',None)

%matplotlib widget 
# %matplotlib inline
matplotlib.rcParams.update({'figure.autolayout': True})

SMALLER_SIZE = 8
SMALL_SIZE = 12
MEDIUM_SIZE = 16
BIGGER_SIZE = 20

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALLER_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

In [ ]:
SCALE_100X = 15.8 # pix/µm 

## Class & functions

In [ ]:
class MultiBeadsDeptho:
   
    def __init__(self, I, scale):
        nz, ny, nx = I.shape[0], I.shape[1], I.shape[2]
        
        self.__I = I
        self.__scale = scale
        self.__I_labeled = np.zeros([nz, ny, nx])
        self.__nz = nz
        self.__ny = ny
        self.__nx = nx
        self.__beadList = []
        self.__nB = 0
        self.__nBvalid = 0
        self.__z_max = 0
        self.__summaryTable = pd.DataFrame({})
        
    def build_beadList(self, plot = 0):
        maxVal = np.zeros(self.__nz)
        for z in range(self.__nz):
            maxVal[z] = np.max(self.__I[z])
        z_max = np.argmax(maxVal)
        self.__z_max = z_max
        I_max = self.__I[z_max]
        threshold = filters.threshold_otsu(I_max)
        I_max_bin = (I_max > threshold)
        I_labeled, nB = measure.label(I_max_bin, return_num = True)
        self.__I_labeled, self.__nB = I_labeled, nB
        
        
        if plot >= 1:
            pStart, pStop = np.percentile(I[self.__z_max], (1, 99))
            I_max_rescaled = exposure.rescale_intensity(I[self.__z_max], in_range=(pStart, pStop))
            figI, axI = plt.subplots(1,1, figsize = (15,5))
            axI.imshow(I_max_rescaled, cmap = 'gray')
        
        # Building __beadList
        for k in range(1,self.__nB+1):
            ym0, xm0 = ndi.center_of_mass(I_max, I_labeled, k)
            bD = BeadDeptho(I, xm0, ym0, self.__scale)
            bD.build_ROI(plot)
            
            if bD.validBead:
#                 print('Job done for the bead in x0, y0 = {:.2f}, {:.2f}'.format(bD.xm0, bD.ym0))
                self.__nBvalid += 1
                bD.build_zProfiles(plot)
                self.__beadList.append(bD)
                
#             else:
#                 print('Not acceptable bead in x0, y0 = {:.2f}, {:.2f}'.format(bD.xm0, bD.ym0))
            
            if plot >= 1:
                if bD.validBead:
                    axI.plot([bD.XYm[0, self.__z_max]], [bD.XYm[1, self.__z_max]], 'c+')
                    axI.text(bD.XYm[0, self.__z_max] + 5, bD.XYm[1, self.__z_max] - 5, str(bD.D), c = 'c')
                else:
                    axI.plot([bD.xm0], [bD.ym0], 'r+')
            
#             print('\n')
            
        # Building __summaryTable
        d = {'D' : [], 'x' : [], 'y' : []}
        otherCols = [c for c in self.__beadList[0].ZfocusDict.keys()]
        for c in otherCols:
            d[c] = []
        for k in range(1,self.__nBvalid+1):
            bD = self.__beadList[k-1]
            d['D'].append(bD.D)
            d['x'].append(bD.XYm[0,bD.z_max])
            d['y'].append(bD.XYm[1,bD.z_max])
            for c in otherCols:
                d[c].append(bD.ZfocusDict[c])        
        self.__summaryTable = pd.DataFrame(d)
                
    def get_beadList(self):
        return(self.__beadList)
          
    def get_summaryTable(self):
        return(self.__summaryTable)
    
    def get_summaryStats(self):
#         print(self.__summaryTable)
        df = self.__summaryTable.drop(['x', 'y'], axis = 1)
        group = df.groupby(df.D)
        df_stats = group.agg(['mean', 'std'])
        df_stats2 = group.agg(['count'])[['Zm3D']].rename(columns = {'Zm3D' : 'count'})
#         print(df_stats2)
        df_stats['count'] = df_stats2['count']
        return(df_stats)
    
    
    @property
    def I(self):
        return(self.__I)
    
    @I.setter
    def I(self, I):
#         self.__I  =  I
        print('FORBIDEN: You cannot change the image after this object instantiation.')
    
    @property
    def nz(self):
        return(self.__nz)
    
    @property
    def ny(self):
        return(self.__ny)
    
    @property
    def nx(self):
        return(self.__nx)
    
    @property
    def nB(self):
        return(self.__nB)

In [ ]:
class BeadDeptho:
    def __init__(self, I, xm0, ym0, scale):
        
        nz, ny, nx = I.shape[0], I.shape[1], I.shape[2]
        
        self.I = I
        self.nz = nz
        self.ny = ny
        self.nx = nx
        self.scale = scale
        self.xm0 = xm0
        self.ym0 = ym0
        
        self.D = 0
        self.threshold = 0
        self.I_cleanROI = np.array([])
        self.XYm = np.zeros((2, self.nz), dtype = int)
        self.validBead = True
        
        self.z_max = 0
        self.validSlice = np.zeros(nz, dtype = bool)
        self.zFirst = 0
        self.zLast = nz
        self.validDepth = nz
        
        self.depthosDict = {}
        self.profileDict = {}
        self.ZfocusDict = {}
    
    
    def build_ROI(self, plot = 0):
        roughSize = np.floor(5*self.scale)
        roughSize += roughSize%2
        x1_roughROI = int(np.floor(self.xm0) - roughSize*0.5) - 1
        x2_roughROI = int(np.floor(self.xm0) + roughSize*0.5)
        y1_roughROI = int(np.floor(self.ym0) - roughSize*0.5) - 1
        y2_roughROI = int(np.floor(self.ym0) + roughSize*0.5)

        if min([x1_roughROI,self.nx-x2_roughROI,y1_roughROI,self.ny-y2_roughROI]) < 0:
            self.validBead = False
            
        if self.validBead:
            I_roughRoi = I[:, y1_roughROI:y2_roughROI, x1_roughROI:x2_roughROI]
            maxVal = np.zeros(self.nz)
            for z in range(self.nz):
                maxVal[z] = np.max(I_roughRoi[z])
            z_max = np.argmax(maxVal)
            self.z_max = z_max

#             print('Zmax = {:.0f}'.format(z_max))
#             figh, axh = plt.subplots(1,2, figsize = (8,4))
#             axh[0].imshow(I_roughRoi[z_max], cmap = 'gray')
#             axh[1].hist(I_roughRoi[z_max].ravel(), bins=256, histtype='step', color='black')
            
            

            counts, binEdges = np.histogram(I_roughRoi[z_max].ravel(), bins=256)

            peaks, peaksProp = find_peaks(counts, height=100, threshold=None, distance=None, prominence=None, \
                               width=None, wlen=None, rel_height=0.5, plateau_size=None)
            
            peakThreshVal = 1200
            
            if counts[peaks[0]] > peakThreshVal:
                self.D = 4.5
                roughSize = np.floor(2*self.D*self.scale)
                roughSize += roughSize%2
                x1_roughROI = int(np.floor(self.xm0) - roughSize*0.5) - 1
                x2_roughROI = int(np.floor(self.xm0) + roughSize*0.5)
                y1_roughROI = int(np.floor(self.ym0) - roughSize*0.5) - 1
                y2_roughROI = int(np.floor(self.ym0) + roughSize*0.5)
                
            else:
                self.D = 2.7
                roughSize = np.floor(2*self.D*self.scale)
                roughSize += roughSize%2
                x1_roughROI = int(np.floor(self.xm0) - roughSize*0.5) - 1
                x2_roughROI = int(np.floor(self.xm0) + roughSize*0.5)
                y1_roughROI = int(np.floor(self.ym0) - roughSize*0.5) - 1
                y2_roughROI = int(np.floor(self.ym0) + roughSize*0.5)
                
                
#             print('First peak height is {:.0f}'.format(counts[peaks[0]]))    
#             print('Detected D is {:.1f}'.format(self.D))
            
            if min([x1_roughROI,self.nx-x2_roughROI,y1_roughROI,self.ny-y2_roughROI]) < 0:
                self.validBead = False
                x1_roughROI, x2_roughROI = max(0, x1_roughROI), min(self.nx, x2_roughROI)
                y1_roughROI, y2_roughROI = max(0, y1_roughROI), min(self.nx, y2_roughROI) 
            
            I_roughRoi = I[:, y1_roughROI:y2_roughROI, x1_roughROI:x2_roughROI]
            
    #         h_roughROI, w_roughROI = x2_roughROI-x1_roughROI, y2_roughROI-y1_roughROI
    #         print(h_roughROI//2, w_roughROI//2)
            
            if self.validBead:
                factorT = 0.8*(self.D == 4.5) + 0.6*(self.D == 2.7)
                threshold = factorT*filters.threshold_otsu(I_roughRoi)
                self.threshold = threshold
                I_roughRoi_binary = (I_roughRoi > threshold)
                I_roughRoi_binary_fh = ndi.binary_fill_holes(I_roughRoi_binary).astype(int)
                Ilab, nB_ROI = measure.label(I_roughRoi_binary_fh[z_max], return_num = True)
                if nB_ROI > 1:
                    self.validBead = False
                    
                if not self.validBead:
                    
                    if plot >= 3:
                        fig, ax = plt.subplots(1,1, figsize = (4,4))
                        ax.imshow(I_roughRoi[z_max], cmap = 'gray')
                
                if self.validBead:
                    area, circ = np.zeros(self.nz), np.zeros(self.nz)
                    for i in range(self.nz):
                        props = measure.regionprops_table(I_roughRoi_binary_fh[i], properties=('perimeter', 'area'))
                        if not np.sum(I_roughRoi_binary_fh[i]) == 0 and props['perimeter'][0] != 0:
                            area[i] = props['area'][0]
                            circ[i] = (4 * np.pi * props['area']) / (props['perimeter'] * props['perimeter'])[0]       
                    
                    validSlice = np.array((area > 50) & (circ > 0.8))
                    zFirst = np.argmax(validSlice)
                    zLast = self.nz - np.argmax(validSlice[::-1])

                    XYm = np.zeros((2, self.nz), dtype = int)
                    cleanSize = int(round(1*self.D*self.scale))
                    cleanSize += 1 + cleanSize%2
                    
                    I_cleanROI = np.zeros([self.nz, cleanSize, cleanSize])

                    try:
                        for i in range(zFirst, zLast):
                            Ym, Xm = ndi.center_of_mass(I_roughRoi[i], labels=I_roughRoi_binary_fh[i], index=1)
                            XYm[1, i], XYm[0, i] = Ym + y1_roughROI, Xm + x1_roughROI
                            roughCenter = int((roughSize+1)//2)
                            tform = transform.EuclideanTransform(rotation=0, \
                                                                 translation = (Xm-roughCenter, Ym-roughCenter))
                            I_tmp = transform.warp(I_roughRoi[i], tform, order = 1, preserve_range = True)

                            I_cleanROI[i] = np.copy(I_tmp[roughCenter-cleanSize//2:roughCenter+cleanSize//2+1,\
                                                          roughCenter-cleanSize//2:roughCenter+cleanSize//2+1])

                        self.validSlice = validSlice
                        self.zFirst = zFirst
                        self.zLast = zLast
                        self.validDepth = zLast-zFirst
                        self.XYm = XYm
                        self.I_cleanROI = I_cleanROI.astype(np.uint16)

                        # VISUALISE
                        if plot >= 2:
    #                                 print('Zfirst = {:.0f} and Zlast = {:.0f}'.format(zFirst, zLast))
                            i = z_max
                            fig, ax = plt.subplots(1,4, figsize = (16,4))

                            pStart, pStop = np.percentile(I[z_max], (1, 99))
                            I_max_rescaled = exposure.rescale_intensity(I[z_max], in_range=(pStart, pStop))
                            ax[0].imshow(I_max_rescaled, cmap = 'gray')
                            ax[0].plot([x1_roughROI,x1_roughROI],[y1_roughROI,y2_roughROI], 'c--')
                            ax[0].plot([x1_roughROI,x2_roughROI],[y2_roughROI,y2_roughROI], 'c--')
                            ax[0].plot([x2_roughROI,x2_roughROI],[y1_roughROI,y2_roughROI], 'c--')
                            ax[0].plot([x1_roughROI,x2_roughROI],[y1_roughROI,y1_roughROI], 'c--')

                            ax[1].imshow(I_roughRoi[i], cmap = 'gray')
                            mid = I_roughRoi[i].shape[0]//2
                            ax[1].plot([0,2*mid],[mid, mid], 'r--', lw = 0.5)
                            ax[1].plot([mid, mid],[0,2*mid], 'r--', lw = 0.5)
                            I_max_binary = (I_roughRoi[i] > threshold)
                            y, x = ndi.center_of_mass(I_roughRoi[i], labels=I_max_binary, index=1)
                            ax[1].plot([x],[y], 'b+')

                            ax[2].imshow(I_tmp, cmap = 'gray')
                            mid = I_tmp.shape[0]//2
                            ax[2].plot([0,2*mid],[mid, mid], 'r--', lw = 0.5)
                            ax[2].plot([mid, mid],[0,2*mid], 'r--', lw = 0.5)
                            I_tmp_binary = (I_tmp > threshold)
                            y, x = ndi.center_of_mass(I_tmp, labels=I_tmp_binary, index=1)
                            ax[2].plot([x],[y], 'b+')

                            ax[3].imshow(I_cleanROI[i], cmap = 'gray')
                            mid = I_cleanROI[i].shape[0]//2
                            ax[3].plot([0,2*mid],[mid, mid], 'r--', lw = 0.5)
                            ax[3].plot([mid, mid],[0,2*mid], 'r--', lw = 0.5)
                            I_cleanRoi_binary = (I_cleanROI > threshold)
                            y, x = ndi.center_of_mass(I_cleanROI[i], labels=I_cleanRoi_binary[i], index=1)
                            ax[3].plot([x],[y], 'b+')

                        
                            
#                             ax[3].hist(I_roughRoi[i].ravel(), bins=256, histtype='step', color='black')
#                             ax[3].plot([0,65535], [peakThreshVal, peakThreshVal], 'r--', lw = 0.8, label = 'D = {:.1f}um'.format(self.D))
#                             ax[3].legend()




                    except:
                        print('BUG ! for the bead in x0, y0 = {:.2f}, {:.2f}'.format(self.xm0, self.ym0))

                     
    def build_zProfiles(self, plot = 0):
        side_ROI = self.I_cleanROI.shape[1]
        mid_ROI = side_ROI//2
        nbPixToAvg = 3 # Have to be an odd number
        deptho_v = np.zeros([self.nz, side_ROI], dtype = np.float64)
        deptho_h = np.zeros([self.nz, side_ROI], dtype = np.float64)
        
        for z in range(self.zFirst, self.zLast):
            templine = side_ROI
            deptho_v[z] = self.I_cleanROI[z,:,mid_ROI] * (1/nbPixToAvg)
#             print(self.I_cleanROI[z,:,mid_ROI])
#             print(1/nbPixToAvg)
#             print(self.deptho_v[z])
            deptho_h[z] = self.I_cleanROI[z,mid_ROI,:] * (1/nbPixToAvg)
    
            for i in range(1, 1 + nbPixToAvg//2):
#                 print(self.I_cleanROI[z,:,mid_ROI - i] * (1/nbPixToAvg) )
                deptho_v[z] += self.I_cleanROI[z,:,mid_ROI - i] * (1/nbPixToAvg) 
                deptho_v[z] += self.I_cleanROI[z,:,mid_ROI + i] * (1/nbPixToAvg)
                
                deptho_h[z] += self.I_cleanROI[z,mid_ROI - i,:] * (1/nbPixToAvg) 
                deptho_h[z] += self.I_cleanROI[z,mid_ROI + i,:] * (1/nbPixToAvg)
          
        deptho_v = deptho_v.astype(np.uint16)
        deptho_h = deptho_h.astype(np.uint16)
        
        self.depthosDict['deptho_v'] = deptho_v
        self.depthosDict['deptho_h'] = deptho_h
        
        
        # 3D caracterisation
        I_binary = np.zeros([self.I_cleanROI.shape[0], self.I_cleanROI.shape[1], self.I_cleanROI.shape[2]])
        I_binary[self.zFirst:self.zLast] = (self.I_cleanROI[self.zFirst:self.zLast] > self.threshold)
        Zm3D, Ym3D, Xm3D = ndi.center_of_mass(self.I_cleanROI, labels=I_binary, index=1)
        self.ZfocusDict['Zm3D'] = Zm3D
        
        
        # Raw profiles
        
        Z = np.array([z for z in range(self.I_cleanROI.shape[0])])
        intensity_tot = np.array([np.sum(self.I_cleanROI[z][I_binary[z].astype(bool)])/(1+np.sum(I_binary[z])) for z in range(self.I_cleanROI.shape[0])]).astype(np.float64)
        intensity_v = np.array([np.sum(deptho_v[z,:])/side_ROI for z in range(deptho_v.shape[0])]).astype(np.float64)
        intensity_h = np.array([np.sum(deptho_h[z,:])/side_ROI for z in range(deptho_h.shape[0])]).astype(np.float64)
        
        Zm_v, Zm_h, Zm_tot = np.argmax(intensity_v), np.argmax(intensity_h), np.argmax(intensity_tot)
        
        self.profileDict['intensity_v'] = intensity_v
        self.profileDict['intensity_h'] = intensity_h
        self.profileDict['intensity_tot'] = intensity_tot
        self.ZfocusDict['Zm_v'] = Zm_v
        self.ZfocusDict['Zm_h'] = Zm_h
        self.ZfocusDict['Zm_tot'] = Zm_tot
        

        # Smoothed profiles
        
        Z_hd = np.arange(0,self.I_cleanROI.shape[0],0.2)
        intensity_v_hd = np.interp(Z_hd, Z, intensity_v)
        intensity_h_hd = np.interp(Z_hd, Z, intensity_h)
        intensity_tot_hd = np.interp(Z_hd, Z, intensity_tot)
        intensity_v_smooth = savgol_filter(intensity_v_hd, 101, 7)
        intensity_h_smooth = savgol_filter(intensity_h_hd, 101, 7)
        intensity_tot_smooth = savgol_filter(intensity_tot_hd, 101, 7)
        
        Zm_v_hd, Zm_h_hd, Zm_tot_hd = Z_hd[np.argmax(intensity_v_smooth)], Z_hd[np.argmax(intensity_h_smooth)], Z_hd[np.argmax(intensity_tot_smooth)]
        
        self.profileDict['intensity_v_smooth'] = intensity_v_smooth 
        self.profileDict['intensity_h_smooth'] = intensity_h_smooth
        self.profileDict['intensity_tot_smooth'] = intensity_tot_smooth
        self.ZfocusDict['Zm_v_hd'] = Zm_v_hd 
        self.ZfocusDict['Zm_h_hd'] = Zm_h_hd
        self.ZfocusDict['Zm_tot_hd'] = Zm_tot_hd
        
        # VISUALISE
        if plot >= 2:
#             fig, ax = plt.subplots(1,3, figsize = (12, 4))
#             ax[0].plot(Z, intensity_v)
#             ax[1].plot(Z, intensity_h)
#             ax[2].plot(Z, (intensity_tot))
#             ax[0].plot([Zm_v, Zm_v], [0, ax[0].get_ylim()[1]], 'r--', lw = 0.8, label = 'Zm_v = {:.2f}'.format(Zm_v))
#             ax[1].plot([Zm_h, Zm_h], [0, ax[1].get_ylim()[1]], 'r--', lw = 0.8, label = 'Zm_h = {:.2f}'.format(Zm_h))
#             ax[2].plot([Zm_tot, Zm_tot], [0, ax[2].get_ylim()[1]], 'r--', lw = 0.8, label = 'Zm_tot = {:.2f}'.format(Zm_tot))
#             ax[0].legend(loc = 'lower right')
#             ax[1].legend(loc = 'lower right')
#             ax[2].legend(loc = 'lower right')

            fig, ax = plt.subplots(1,3, figsize = (12, 4))
            ax[0].plot(Z, intensity_v, 'b-')
            ax[1].plot(Z, intensity_h, 'b-')
            ax[2].plot(Z, (intensity_tot), 'b-')
            ax[0].plot(Z_hd, intensity_v_smooth, 'k--')
            ax[1].plot(Z_hd, intensity_h_smooth, 'k--')
            ax[2].plot(Z_hd, intensity_tot_smooth, 'k--')
            ax[0].plot([Zm_v_hd, Zm_v_hd], [0, ax[0].get_ylim()[1]], 'r--', lw = 0.8, label = 'Zm_v_hd = {:.2f}'.format(Zm_v_hd))
            ax[1].plot([Zm_h_hd, Zm_h_hd], [0, ax[1].get_ylim()[1]], 'r--', lw = 0.8, label = 'Zm_h_hd = {:.2f}'.format(Zm_h_hd))
            ax[2].plot([Zm_tot_hd, Zm_tot_hd], [0, ax[2].get_ylim()[1]], 'r--', lw = 0.8, label = 'Zm_tot_hd = {:.2f}'.format(Zm_tot_hd))
            ax[0].legend(loc = 'lower right')
            ax[1].legend(loc = 'lower right')
            ax[2].legend(loc = 'lower right')

            print('Zm_v = {:.2f}, Zm_h = {:.2f}, Zm_tot = {:.2f}'\
                  .format(Zm_v, Zm_h, Zm_tot))
            print('Zm_v_hd = {:.2f}, Zm_h_hd = {:.2f}, Zm_tot_hd = {:.2f}'\
                  .format(Zm_v_hd, Zm_h_hd, Zm_tot_hd))
            print('Xm3D = {:.2f}, Ym3D = {:.2f}, Zm3D = {:.2f}'\
                  .format(Xm3D, Ym3D, Zm3D))

## Scripts

### Chambre 2

#### Global

In [ ]:
# testDataFolder = 'D://PMMH_ComputerData//Matlab Analysis//Data_Joseph//Raw//20.06.15_Deptho//depthoAnalysis_test'
testDataFolder = 'C://Users//JosephVermeil//Desktop//test_AnalyseDepthos'
listDepthos = [f for f in os.listdir(testDataFolder) if (os.path.isfile(os.path.join(testDataFolder, f)) and f.endswith(".tif"))]
listDepthos

In [ ]:
f = listDepthos[0]
filePath = os.path.join(testDataFolder, f)
I = io.imread(filePath)
mbd = MultiBeadsDeptho(I, SCALE_100X)
mbd.build_beadList(plot = 1)
listBeads = mbd.get_beadList()
for bD in listBeads:
    if bD.validBead:
        print('{:.0f} valid slices ; Zfirst = {:.0f} and Zlast = {:.0f}'.format(bD.validDepth, bD.zFirst, bD.zLast))
        fig, ax = plt.subplots(1,2, figsize = (8, 5))
        
        dV = bD.depthosDict['deptho_v']
        pStart, pStop = np.percentile(dV, (1, 99))
        dV_rescaled = exposure.rescale_intensity(dV, in_range=(pStart, pStop))
        ax[0].imshow(dV_rescaled, cmap='gray')
        ax[0].set_ylim([bD.zLast, bD.zFirst])
        
        dH = bD.depthosDict['deptho_h']
        pStart, pStop = np.percentile(dH, (1, 99))
        dH_rescaled = exposure.rescale_intensity(dH, in_range=(pStart, pStop))
        ax[1].imshow(dH_rescaled, cmap='gray')
        ax[1].set_ylim([bD.zLast, bD.zFirst])

In [ ]:
df = pd.DataFrame({})

for f in listDepthos:
    filePath = os.path.join(testDataFolder, f)
    I = io.imread(filePath)
    mbd = MultiBeadsDeptho(I, SCALE_100X)
    mbd.build_beadList(plot = 0)
    df_stats = mbd.get_summaryStats()
#     print(df_stats)
    if df_stats['count'].values[0] >= 2 and df_stats['count'].values[1] >= 2:
        S = df_stats.loc[df_stats.index == 2.7,(slice(None), ['mean'])]
        L = df_stats.loc[df_stats.index == 4.5,(slice(None), ['mean'])]
        new_row = L.sub(S.values, axis='rows')
        new_row.index = [f]
        if df.shape == (0, 0):
            df = new_row
        else:
            df = pd.concat([df, new_row], axis = 0)

df

#### multiDeptho2

In [ ]:
f = listDepthos[1]
print(f)
filePath = os.path.join(testDataFolder, f)
I = io.imread(filePath)
mbd = MultiBeadsDeptho(I, SCALE_100X)
mbd.build_beadList(plot = 2)
mbd.get_summaryStats()

#### multiDeptho3

In [ ]:
f = listDepthos[2]
print(f)
filePath = os.path.join(testDataFolder, f)
I = io.imread(filePath)
mbd = MultiBeadsDeptho(I, SCALE_100X)
mbd.build_beadList(plot = 2)
mbd.get_summaryStats()

#### multiDeptho4

In [ ]:
f = listDepthos[3]
print(f)
filePath = os.path.join(testDataFolder, f)
I = io.imread(filePath)
mbd = MultiBeadsDeptho(I, SCALE_100X)
mbd.build_beadList(plot = 2)
mbd.get_summaryStats()

#### multiDeptho5

In [ ]:
f = listDepthos[4]
print(f)
filePath = os.path.join(testDataFolder, f)
I = io.imread(filePath)
mbd = MultiBeadsDeptho(I, SCALE_100X)
mbd.build_beadList(plot = 2)
mbd.get_summaryStats()

#### multiDeptho6

In [ ]:
f = listDepthos[5]
print(f)
filePath = os.path.join(testDataFolder, f)
I = io.imread(filePath)
mbd = MultiBeadsDeptho(I, SCALE_100X)
mbd.build_beadList(plot = 2)
mbd.get_summaryStats()

#### multiDeptho7

In [ ]:
f = listDepthos[6]
print(f)
filePath = os.path.join(testDataFolder, f)
I = io.imread(filePath)
mbd = MultiBeadsDeptho(I, SCALE_100X)
mbd.build_beadList(plot = 2)
mbd.get_summaryStats()

#### multiDeptho8

In [ ]:
f = listDepthos[7]
print(f)
filePath = os.path.join(testDataFolder, f)
I = io.imread(filePath)
mbd = MultiBeadsDeptho(I, SCALE_100X)
mbd.build_beadList(plot = 2)
mbd.get_summaryStats()

#### multiDeptho9

In [ ]:
f = listDepthos[8]
print(f)
filePath = os.path.join(testDataFolder, f)
I = io.imread(filePath)
mbd = MultiBeadsDeptho(I, SCALE_100X)
mbd.build_beadList(plot = 2)
mbd.get_summaryStats()

#### multiDeptho10

In [ ]:
f = listDepthos[0]
print(f)
filePath = os.path.join(testDataFolder, f)
I = io.imread(filePath)
mbd = MultiBeadsDeptho(I, SCALE_100X)
mbd.build_beadList(plot = 2)
mbd.get_summaryStats()

### Autres

In [ ]:
plt.close('all')

## Old code & tests

In [ ]:
testDataFolder = 'C://Users//JosephVermeil//Desktop//test_AnalyseDepthos'
listDepthos = [f for f in os.listdir(testDataFolder) if (os.path.isfile(os.path.join(testDataFolder, f)) and f.endswith(".tif"))]

fileName = listDepthos[2]
filePath = os.path.join(testDataFolder,fileName)
I = io.imread(filePath)

nz, ny, nx = I.shape[0], I.shape[1], I.shape[2]
maxVal = np.zeros(nz)
for z in range(nz):
    maxVal[z] = np.max(I[z])

    
#
Z = np.array([i for i in range(nz)])
fig, ax = plt.subplots(1,1, figsize = (8,4))
ax.plot(Z, maxVal)
fig.show()
#


zMax = np.argmax(maxVal)
IMax = I[zMax]
#
fig, ax = plt.subplots(3,1, figsize = (9,9))
ax[0].imshow(IMax, cmap = 'gray')
ax[0].set_title('Raw')
#
#
threshold = filters.threshold_li(IMax)
binIMax = IMax > threshold
ax[1].imshow(binIMax, cmap = 'gray')
ax[1].set_title('Filter: Li')
#
#
threshold = filters.threshold_otsu(IMax)
binIMax = IMax > threshold
ax[2].imshow(binIMax, cmap = 'gray')
ax[2].set_title('Filter: Otsu')
#
fig.show()

In [ ]:
# df1 = pd.DataFrame({})

# f = listDepthos[0]
# filePath = os.path.join(testDataFolder, f)
# I = io.imread(filePath)
# mbd = MultiBeadsDeptho(I, SCALE_100X)
# mbd.build_beadList(plot = 1)
# listBeads = mbd.get_beadList()
# if df1.shape == (0, 0):
#     df1 = mbd.get_summaryStats()
# else:
#     df1 = pd.concat([df1, mbd.get_summaryTable()], axis = 0).reset_index()

# df1

In [ ]:
# if not I.ndim == 3:
#     print('Unexpected image format !')
# else:
#     I0 = I[0]

# fig, ax = plt.subplots(1,1)
# ax.imshow(I[2], cmap = 'gray')
# fig.tight_layout()
# fig.show()

# I.shape
# smallI = I[198:203]
# smallROI = I[:,100:400,100:500]
# # plt.imshow(smallROI, cmap = 'gray')
# threshold = skimage.filters.threshold_otsu(smallROI)
# threshold

# fig, ax = plt.subplots(1,1)
# ax.hist(smallROI.ravel(), bins=256, histtype='step', color='black')
# fig.show()

# np.zeros([nz, ny, nx])

In [ ]:
# Tests with numpy arrays

# A = np.array([[1, 2, 3],[4, 5, 6],[7, 8, 9]])
# np.sum(A[0])
# O = np.zeros([3, 2, 2])
# O[1,:,:] = A[:2,:2]
# O[2,:,:] = A[1:,1:]
# b = np.array([1, 0], dtype = bool)
# O[0][b]